# Risk Mediation for Heart Failure for BMI to FPG and SBP

This notebook is designed to calculate the mediation factors for heart failure for FPG and SBP. LDL-C does not directly affect HF in our model, and so does not have a mediation factor here. We also test if we need to include the risk mediation of BMI on FPG based on the results of a trial and the observed effect on FPG.

## First, we will load in the relative risk values from GBD

In [1]:
from db_queries import get_outputs as go
from db_queries import get_ids
from get_draws.api import get_draws
import numpy as np
import pandas as pd

In [2]:
ihd_bmi = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=370, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

ihd_bmi = ihd_bmi.loc[ihd_bmi.cause_id==493].reset_index()
ihd_bmi = ihd_bmi[['age_group_id','sex_id','draw_0']]
ihd_bmi['age_start'] = np.where(ihd_bmi['age_group_id']==10,25,np.where(ihd_bmi['age_group_id']==11,30,np.where(ihd_bmi['age_group_id']==12,35,np.where(ihd_bmi['age_group_id']==13,40,np.where(ihd_bmi['age_group_id']==14,45,np.where(ihd_bmi['age_group_id']==15,50,np.where(ihd_bmi['age_group_id']==16,55,np.where(ihd_bmi['age_group_id']==17,60,np.where(ihd_bmi['age_group_id']==18,65,np.where(ihd_bmi['age_group_id']==19,70,np.where(ihd_bmi['age_group_id']==20,75,np.where(ihd_bmi['age_group_id']==30,80,np.where(ihd_bmi['age_group_id']==31,85,np.where(ihd_bmi['age_group_id']==32,90,0))))))))))))))
ihd_bmi['sex'] = np.where(ihd_bmi['sex_id']==1,'Male','Female')
ihd_bmi = ihd_bmi.rename(columns={"draw_0": "ihd_bmi_rr"}).reset_index()
ihd_bmi = ihd_bmi.drop_duplicates(subset=['age_start', 'sex']).reset_index()
ihd_bmi.head()

/ihme/homes/lutzes/.conda/envs/vivarium_nih_us_cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id


,level_0,index,age_group_id,sex_id,ihd_bmi_rr,age_start,sex
0,0,0,19,1,1.296208,70,Male
1,1,1,20,1,1.291149,75,Male
2,2,2,18,1,1.395200,65,Male
3,3,3,16,1,1.425974,55,Male
4,4,4,11,1,2.094858,30,Male


In [3]:
stroke_bmi = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=370, 
                    source= 'rr', 
                    year_id=2019, 
                    age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                    sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

stroke_bmi = stroke_bmi.loc[stroke_bmi.cause_id==495].reset_index()
stroke_bmi = stroke_bmi[['age_group_id','sex_id','draw_0']]
stroke_bmi['age_start'] = np.where(stroke_bmi['age_group_id']==10,25,np.where(stroke_bmi['age_group_id']==11,30,np.where(stroke_bmi['age_group_id']==12,35,np.where(stroke_bmi['age_group_id']==13,40,np.where(stroke_bmi['age_group_id']==14,45,np.where(stroke_bmi['age_group_id']==15,50,np.where(stroke_bmi['age_group_id']==16,55,np.where(stroke_bmi['age_group_id']==17,60,np.where(stroke_bmi['age_group_id']==18,65,np.where(stroke_bmi['age_group_id']==19,70,np.where(stroke_bmi['age_group_id']==20,75,np.where(stroke_bmi['age_group_id']==30,80,np.where(stroke_bmi['age_group_id']==31,85,np.where(stroke_bmi['age_group_id']==32,90,0))))))))))))))
stroke_bmi['sex'] = np.where(stroke_bmi['sex_id']==1,'Male','Female')
stroke_bmi = stroke_bmi.rename(columns={"draw_0": "stroke_bmi_rr"}).reset_index()
stroke_bmi = stroke_bmi.drop_duplicates(subset=['age_start', 'sex']).reset_index()
stroke_bmi.head()

/ihme/homes/lutzes/.conda/envs/vivarium_nih_us_cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id


,level_0,index,age_group_id,sex_id,stroke_bmi_rr,age_start,sex
0,0,0,12,1,1.919950,35,Male
1,1,1,15,1,1.727334,50,Male
2,2,2,14,1,1.809969,45,Male
3,3,3,17,1,1.497946,60,Male
4,4,4,10,1,2.405075,25,Male


In [4]:
ihd_fpg = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=105, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

ihd_fpg = ihd_fpg.loc[ihd_fpg.cause_id==493].reset_index()
ihd_fpg = ihd_fpg[['age_group_id','sex_id','draw_0']]
ihd_fpg['age_start'] = np.where(ihd_fpg['age_group_id']==10,25,np.where(ihd_fpg['age_group_id']==11,30,np.where(ihd_fpg['age_group_id']==12,35,np.where(ihd_fpg['age_group_id']==13,40,np.where(ihd_fpg['age_group_id']==14,45,np.where(ihd_fpg['age_group_id']==15,50,np.where(ihd_fpg['age_group_id']==16,55,np.where(ihd_fpg['age_group_id']==17,60,np.where(ihd_fpg['age_group_id']==18,65,np.where(ihd_fpg['age_group_id']==19,70,np.where(ihd_fpg['age_group_id']==20,75,np.where(ihd_fpg['age_group_id']==30,80,np.where(ihd_fpg['age_group_id']==31,85,np.where(ihd_fpg['age_group_id']==32,90,0))))))))))))))
ihd_fpg['sex'] = np.where(ihd_fpg['sex_id']==1,'Male','Female')
ihd_fpg = ihd_fpg.rename(columns={"draw_0": "ihd_fpg_rr"}).reset_index()
ihd_fpg = ihd_fpg.drop_duplicates(subset=['age_start', 'sex']).reset_index()
ihd_fpg.head()

/ihme/homes/lutzes/.conda/envs/vivarium_nih_us_cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id


,level_0,index,age_group_id,sex_id,ihd_fpg_rr,age_start,sex
0,0,0,10,2,1.9325,25,Female
1,1,1,11,2,1.6915,30,Female
2,2,2,12,2,1.4505,35,Female
3,3,3,13,2,1.3135,40,Female
4,4,4,14,2,1.2805,45,Female


In [5]:
stroke_fpg = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=105, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

stroke_fpg = stroke_fpg.loc[stroke_fpg.cause_id==495].reset_index()
stroke_fpg = stroke_fpg[['age_group_id','sex_id','draw_0']]
stroke_fpg['age_start'] = np.where(stroke_fpg['age_group_id']==10,25,np.where(stroke_fpg['age_group_id']==11,30,np.where(stroke_fpg['age_group_id']==12,35,np.where(stroke_fpg['age_group_id']==13,40,np.where(stroke_fpg['age_group_id']==14,45,np.where(stroke_fpg['age_group_id']==15,50,np.where(stroke_fpg['age_group_id']==16,55,np.where(stroke_fpg['age_group_id']==17,60,np.where(stroke_fpg['age_group_id']==18,65,np.where(stroke_fpg['age_group_id']==19,70,np.where(stroke_fpg['age_group_id']==20,75,np.where(stroke_fpg['age_group_id']==30,80,np.where(stroke_fpg['age_group_id']==31,85,np.where(stroke_fpg['age_group_id']==32,90,0))))))))))))))
stroke_fpg['sex'] = np.where(stroke_fpg['sex_id']==1,'Male','Female')
stroke_fpg = stroke_fpg.rename(columns={"draw_0": "stroke_fpg_rr"}).reset_index()
stroke_fpg = stroke_fpg.drop_duplicates(subset=['age_start', 'sex']).reset_index()
stroke_fpg.head()

/ihme/homes/lutzes/.conda/envs/vivarium_nih_us_cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id


,level_0,index,age_group_id,sex_id,stroke_fpg_rr,age_start,sex
0,0,0,10,2,1.5236,25,Female
1,1,1,11,2,1.3974,30,Female
2,2,2,12,2,1.2711,35,Female
3,3,3,13,2,1.2058,40,Female
4,4,4,14,2,1.2012,45,Female


In [6]:
ihd_sbp = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=107, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

ihd_sbp = ihd_sbp.loc[ihd_sbp.cause_id==493].reset_index()
ihd_sbp = ihd_sbp[['age_group_id','sex_id','draw_0']]
ihd_sbp['age_start'] = np.where(ihd_sbp['age_group_id']==10,25,np.where(ihd_sbp['age_group_id']==11,30,np.where(ihd_sbp['age_group_id']==12,35,np.where(ihd_sbp['age_group_id']==13,40,np.where(ihd_sbp['age_group_id']==14,45,np.where(ihd_sbp['age_group_id']==15,50,np.where(ihd_sbp['age_group_id']==16,55,np.where(ihd_sbp['age_group_id']==17,60,np.where(ihd_sbp['age_group_id']==18,65,np.where(ihd_sbp['age_group_id']==19,70,np.where(ihd_sbp['age_group_id']==20,75,np.where(ihd_sbp['age_group_id']==30,80,np.where(ihd_sbp['age_group_id']==31,85,np.where(ihd_sbp['age_group_id']==32,90,0))))))))))))))
ihd_sbp['sex'] = np.where(ihd_sbp['sex_id']==1,'Male','Female')
ihd_sbp = ihd_sbp.rename(columns={"draw_0": "ihd_sbp_rr"}).reset_index()
ihd_sbp = ihd_sbp.drop_duplicates(subset=['age_start', 'sex']).reset_index()
ihd_sbp.head()

/ihme/homes/lutzes/.conda/envs/vivarium_nih_us_cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id


,level_0,index,age_group_id,sex_id,ihd_sbp_rr,age_start,sex
0,0,0,10,1,2.0317,25,Male
1,1,1,11,1,1.8558,30,Male
2,2,2,12,1,1.6799,35,Male
3,3,3,13,1,1.5729,40,Male
4,4,4,14,1,1.5348,45,Male


In [7]:
stroke_sbp = get_draws(gbd_id_type= 'rei_id', 
                    gbd_id=107, 
                    source= 'rr', 
                    year_id=2019, 
                   age_group_id=[10,11,12,13,14,15,16,17,18,19,20,30,31,32,33],
                   sex_id=[1,2],
                    gbd_round_id=6, 
                    status= 'best', 
                    decomp_step= 'step4')

stroke_sbp = stroke_sbp.loc[stroke_sbp.cause_id==493].reset_index()
stroke_sbp = stroke_sbp[['age_group_id','sex_id','draw_0']]
stroke_sbp['age_start'] = np.where(stroke_sbp['age_group_id']==10,25,np.where(stroke_sbp['age_group_id']==11,30,np.where(stroke_sbp['age_group_id']==12,35,np.where(stroke_sbp['age_group_id']==13,40,np.where(stroke_sbp['age_group_id']==14,45,np.where(stroke_sbp['age_group_id']==15,50,np.where(stroke_sbp['age_group_id']==16,55,np.where(stroke_sbp['age_group_id']==17,60,np.where(stroke_sbp['age_group_id']==18,65,np.where(stroke_sbp['age_group_id']==19,70,np.where(stroke_sbp['age_group_id']==20,75,np.where(stroke_sbp['age_group_id']==30,80,np.where(stroke_sbp['age_group_id']==31,85,np.where(stroke_sbp['age_group_id']==32,90,0))))))))))))))
stroke_sbp['sex'] = np.where(stroke_sbp['sex_id']==1,'Male','Female')
stroke_sbp = stroke_sbp.rename(columns={"draw_0": "stroke_sbp_rr"}).reset_index()
stroke_sbp = stroke_sbp.drop_duplicates(subset=['age_start', 'sex']).reset_index()
stroke_sbp.head()

/ihme/homes/lutzes/.conda/envs/vivarium_nih_us_cvd/lib/python3.8/site-packages/get_draws/transforms/version_transforms.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[version_id_type_name] = version_id


,level_0,index,age_group_id,sex_id,stroke_sbp_rr,age_start,sex
0,0,0,10,1,2.0317,25,Male
1,1,1,11,1,1.8558,30,Male
2,2,2,12,1,1.6799,35,Male
3,3,3,13,1,1.5729,40,Male
4,4,4,14,1,1.5348,45,Male


## Now we can calculate the deltas

Note that we are finding the same value (deltas) based on two causes (IHD and stroke). Ideally, these should be identical. We find that they are very similar, making me more confident in these calculations. 

In [8]:
bmi_delta = ihd_bmi.merge(ihd_fpg[['ihd_fpg_rr','age_start','sex']], on=['age_start','sex']).merge(
    ihd_sbp[['ihd_sbp_rr','age_start','sex']], on=['age_start','sex']).merge(
    stroke_bmi[['stroke_bmi_rr','age_start','sex']], on=['age_start','sex']).merge(
    stroke_fpg[['stroke_fpg_rr','age_start','sex']], on=['age_start','sex']).merge(
    stroke_sbp[['stroke_sbp_rr','age_start','sex']], on=['age_start','sex'])
bmi_delta['fpg_delta_1'] = ((0.149278 * (bmi_delta['ihd_bmi_rr'] - 1)) + 1)/ bmi_delta['ihd_fpg_rr']
bmi_delta['sbp_delta_1'] = ((0.311516 * (bmi_delta['ihd_bmi_rr'] - 1)) + 1)/ bmi_delta['ihd_sbp_rr']
bmi_delta['fpg_delta_2'] = ((0.149278 * (bmi_delta['stroke_bmi_rr'] - 1)) + 1)/ bmi_delta['stroke_fpg_rr']
bmi_delta['sbp_delta_2'] = ((0.311516 * (bmi_delta['stroke_bmi_rr'] - 1)) + 1)/ bmi_delta['stroke_sbp_rr']
bmi_delta['delta_diff_fpg'] = bmi_delta['fpg_delta_1'] - bmi_delta['fpg_delta_2']
bmi_delta['delta_diff_sbp'] = bmi_delta['sbp_delta_1'] - bmi_delta['sbp_delta_2']
bmi_delta = bmi_delta[['ihd_bmi_rr','ihd_fpg_rr','ihd_sbp_rr','stroke_bmi_rr','stroke_sbp_rr','stroke_fpg_rr','age_start', 'sex', 'fpg_delta_1','sbp_delta_1', 'fpg_delta_2','sbp_delta_2','delta_diff_fpg','delta_diff_sbp']]
bmi_delta.head()

,ihd_bmi_rr,ihd_fpg_rr,ihd_sbp_rr,stroke_bmi_rr,stroke_sbp_rr,stroke_fpg_rr,age_start,sex,fpg_delta_1,sbp_delta_1,fpg_delta_2,sbp_delta_2,delta_diff_fpg,delta_diff_sbp
0,1.296208,1.1512,1.3402,1.271957,1.3402,1.1760,70,Male,0.907068,0.815008,0.884862,0.809371,0.022207,0.005637
1,1.291149,1.1897,1.2947,1.273751,1.2947,1.1660,75,Male,0.877080,0.842433,0.892680,0.838247,-0.015600,0.004186
2,1.395200,1.1485,1.3821,1.344813,1.3821,1.1833,65,Male,0.922068,0.812612,0.888594,0.801255,0.033474,0.011357
3,1.425974,1.2145,1.4584,1.556205,1.4584,1.1923,55,Male,0.875742,0.776672,0.908353,0.804489,-0.032611,-0.027817
4,2.094858,1.6915,1.8558,2.220086,1.8558,1.3974,30,Male,0.687815,0.722635,0.845951,0.743656,-0.158137,-0.021021


In [9]:
print(bmi_delta['delta_diff_sbp'].mean())
print(bmi_delta['delta_diff_sbp'].max())
print(bmi_delta['delta_diff_fpg'].mean())
print(bmi_delta['delta_diff_fpg'].min())

0.0026496325876054533
0.1695088860694438
-0.06296800931832136
-0.15813653771159408


## The RR for SBP to HF is categorical, we need continuous

The literature value for SBP RR to HF are in categories. To account for this, we found a continuous RR that approximates the same values. The mean_SBP are based on exposure values form GBD. 

In [10]:
SBP_RR = pd.DataFrame(np.array([['<120',1,110.6],['120-129',1.27,124.4],['130-139',1.5,134.2],['140+',1.76,150.7]]),
                   columns=['SBP_Group', 'RR_original', 'Mean_SBP'])
SBP_RR['Mean_SBP'] = pd.to_numeric(SBP_RR['Mean_SBP'])
SBP_RR['Change_in_SBP'] = SBP_RR['Mean_SBP'] - 110.6
cont_RR = 1.016
SBP_RR['New_RR'] = np.power(cont_RR,SBP_RR['Change_in_SBP'])
SBP_RR

,SBP_Group,RR_original,Mean_SBP,Change_in_SBP,New_RR
0,<120,1,110.6,0.0,1.000000
1,120-129,1.27,124.4,13.8,1.244896
2,130-139,1.5,134.2,23.6,1.454426
3,140+,1.76,150.7,40.1,1.889895


## Lastly, we find the Mediation Factors

In [11]:
rr_sbp = 1.016
rr_bmi = 1.14

In [12]:
bmi_delta['sbp_delta_1'] = bmi_delta.sbp_delta_1 * 2 
bmi_delta['MF_SBP_1'] = ((rr_sbp-1)*bmi_delta['sbp_delta_1']) / (rr_bmi - 1)
bmi_delta = bmi_delta[['age_start', 'sex', 'sbp_delta_1','MF_SBP_1']]
bmi_delta.head()

,age_start,sex,sbp_delta_1,MF_SBP_1
0,70,Male,1.630016,0.186288
1,75,Male,1.684865,0.192556
2,65,Male,1.625224,0.185740
3,55,Male,1.553343,0.177525
4,30,Male,1.445270,0.165174


In [13]:
HF_MF=pd.DataFrame(bmi_delta)
HF_MF.to_csv('/ihme/homes/lutzes/heart_failure_MF.csv')

## Testing based on the intervention if we need FPG 

This is bit tough/estimated since right now I am extracting data from a figure. I hope to find better data at some point. I am choosing to assess this at 1 year since that is when there is the greatest change in both BMI and FPG.

In [ ]:
#int_bmi = 2.5
#int_fpg = 0.3 mmol/L --> 5 mg/dL

For BMI will account for a 1/2 unit change in FPG, the intervention saw a ~5 unit change, we should add mediation 